# undate partial date duration check
compare undate interval duration calculation for date rnages between partial dates with Shakespeare and Company Project events dataset

In [1]:
# install from feature branch for now, until merged
%pip install git+https://github.com/dh-tech/undate-python.git@story/3-partially-known-dates

  Cloning https://github.com/dh-tech/undate-python.git (to revision story/3-partially-known-dates) to /tmp/pip-req-build-aoklox4b
  Running command git clone --filter=blob:none --quiet https://github.com/dh-tech/undate-python.git /tmp/pip-req-build-aoklox4b
  Running command git checkout -b story/3-partially-known-dates --track origin/story/3-partially-known-dates
  Switched to a new branch 'story/3-partially-known-dates'
  Branch 'story/3-partially-known-dates' set up to track remote branch 'story/3-partially-known-dates' from 'origin'.
  Resolved https://github.com/dh-tech/undate-python.git to commit 615cb0d9af313128a562107305d1b7c2eddd9535
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for undate: filename=undate-0.2.0.dev0-py3-none-any.whl size=15200 sha256=ca2e25447c84ad830f1e7ac31a43d67701390d594fff44637a565c3de5bb6134
  Stored in directory: /tmp/pip-ephem-wheel-cache-ctzxaxcn/

In [2]:
import pandas as pd

# load most recent version of S&co events dataset

# dataspace link on S&co website is currently broken
#events_df = pd.read_csv("https://dataspace.princeton.edu/bitstream/88435/dsp019306t2441/2/SCoData_events_v1.2_2022-01.csv")
# other dataset link resulting in an error; incomplete download?
# events_df = pd.read_csv("https://dataspace.princeton.edu/bitstream/88435/dsp019306t2441/1")
events_df = pd.read_csv("https://github.com/rlskoeser/shxco-missingdata-specreading/raw/main/data/source-data/SCoData_events_v1.2_2022-01.csv")
events_df.head()

<ipython-input-2-7ea275849855>:9: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  events_df = pd.read_csv("https://github.com/rlskoeser/shxco-missingdata-specreading/raw/main/data/source-data/SCoData_events_v1.2_2022-01.csv")


,event_type,start_date,end_date,member_uris,member_names,member_sort_names,subscription_price_paid,subscription_deposit,subscription_duration,subscription_duration_days,...,item_uri,item_title,item_volume,item_authors,item_year,item_notes,source_type,source_citation,source_manifest,source_image
0,Generic,1920,NaN,https://shakespeareandco.princeton.edu/members...,Raymonde Linossier,"Linossier, Raymonde",NaN,NaN,NaN,NaN,...,https://shakespeareandco.princeton.edu/books/b...,Pigs Is Pigs,NaN,"Butler, Ellis Parker",1906.0,NaN,Lending Library Card,"Sylvia Beach, Raymonde Linossier Lending Libra...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/00...
1,Subscription,1921,NaN,https://shakespeareandco.princeton.edu/members...,Mme Garreta,"Garreta, Mme",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Address Book,"Sylvia Beach, Address Book 1919–1935, box 69, ...",NaN,NaN
2,Borrow,1922,1922-08-23,https://shakespeareandco.princeton.edu/members...,Mr. Rhys,"Rhys, Mr.",NaN,NaN,NaN,NaN,...,https://shakespeareandco.princeton.edu/books/c...,Typhoon,NaN,"Conrad, Joseph",1902.0,NaN,Lending Library Card,"Sylvia Beach, Rhys Lending Library Card, Box 4...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/67...
3,Generic,1922,NaN,https://shakespeareandco.princeton.edu/members...,Ernest Walsh,"Walsh, Ernest",NaN,NaN,NaN,NaN,...,https://shakespeareandco.princeton.edu/books/b...,The Pretty Lady,NaN,"Bennett, Arnold",1918.0,NaN,Lending Library Card,"Sylvia Beach, Ernest Walsh Lending Library Car...",https://figgy.princeton.edu/concern/scanned_re...,https://iiif.princeton.edu/loris/figgy_prod/af...
4,Subscription,1922,NaN,https://shakespeareandco.princeton.edu/members...,Mr. Lincoln,"Lincoln, Mr.",NaN,7.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Address Book,"Sylvia Beach, Address Book 1919–1935, box 69, ...",NaN,NaN


## method to calculate durations

define a method to initialize undate interval from start and end date string in ISO format as used in S&co datasets


**Note:** that there's an off-by-one discrepancy between how we currently calculate duration in undate and in the Shakespeare and Company Project code: the S&co code counts the first day in the range but not the last (could also be thought of as counting half of start and end dates).

In [3]:
from undate.undate import UndateInterval
from undate.dateformat.iso8601 import ISO8601DateFormat

def undate_duration(start_date, end_date):
  isoformat = ISO8601DateFormat()

  unstart = isoformat.parse(start_date)
  unend = isoformat.parse(end_date)
  interval = UndateInterval(earliest=unstart, latest=unend)

  # subtract one here for simplicity of comparison,
  # to reconcile difference between how duration logic

  return interval.duration().days - 1

## subscription events

compare subscription events with known duration

In [4]:
# identify subscription eventss with duration information
subs_duration = events_df[events_df.subscription_duration_days.notna()]
subs_duration.head()

,event_type,start_date,end_date,member_uris,member_names,member_sort_names,subscription_price_paid,subscription_deposit,subscription_duration,subscription_duration_days,...,item_uri,item_title,item_volume,item_authors,item_year,item_notes,source_type,source_citation,source_manifest,source_image
28,Subscription,1927,1928,https://shakespeareandco.princeton.edu/members...,Arthur Elliott Felkin,"Felkin, Arthur Elliott",NaN,NaN,1 year,365.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Address Book,"Sylvia Beach, Address Book 1919–1935, box 69, ...",NaN,NaN
70,Subscription,1931,1932,https://shakespeareandco.princeton.edu/members...,Geraldine Deknatel;William Deknatel,"Deknatel, Geraldine;Deknatel, William",NaN,NaN,1 year,365.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Address Book,"Sylvia Beach, Address Book 1919–1935, box 69, ...",NaN,NaN
233,Subscription,1921-07,1921-08,https://shakespeareandco.princeton.edu/members...,Mrs. G. S. Madam,"Madam, Mrs. G. S.",NaN,NaN,1 month,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Address Book,"Sylvia Beach, Address Book 1919–1935, box 69, ...",NaN,NaN
234,Subscription,1921-09,1922-02,https://shakespeareandco.princeton.edu/members...,Anne Moderwell;Hiram Moderwell / H. K. Moderwell,"Moderwell, Anne;Moderwell, Hiram",NaN,NaN,5 months,153.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Address Book,"Sylvia Beach, Address Book 1919–1935, box 69, ...",NaN,NaN
260,Subscription,1923-06,1923-10,https://shakespeareandco.princeton.edu/members...,Victor Llona,"Llona, Victor",NaN,NaN,4 months,122.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Address Book,"Sylvia Beach, Address Book 1919–1935, box 69, ...",NaN,NaN


In [5]:
# what do the subscription duration day values look like?
subs_duration.subscription_duration_days.value_counts()

31.0     2997
30.0     1975
92.0      936
91.0      397
365.0     337
         ... 
69.0        1
36.0        1
73.0        1
574.0       1
171.0       1
Name: subscription_duration_days, Length: 133, dtype: int64

In [6]:
subs_duration.subscription_duration_days.describe()

count    9146.000000
mean       72.142685
std        81.559368
min         1.000000
25%        30.000000
50%        31.000000
75%        91.000000
max       574.000000
Name: subscription_duration_days, dtype: float64

In [7]:
# do we have unknown start/end date values?
subs_duration[subs_duration.start_date.isna()]

,event_type,start_date,end_date,member_uris,member_names,member_sort_names,subscription_price_paid,subscription_deposit,subscription_duration,subscription_duration_days,...,item_uri,item_title,item_volume,item_authors,item_year,item_notes,source_type,source_citation,source_manifest,source_image


In [8]:
subs_duration[subs_duration.end_date.isna()]

,event_type,start_date,end_date,member_uris,member_names,member_sort_names,subscription_price_paid,subscription_deposit,subscription_duration,subscription_duration_days,...,item_uri,item_title,item_volume,item_authors,item_year,item_notes,source_type,source_citation,source_manifest,source_image
13168,Subscription,1932-10-06,NaN,https://shakespeareandco.princeton.edu/members...,Jean (Bakewell) Connolly / Mrs. Cyril Connolly,"Connolly, Jean",NaN,100.0,NaN,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Logbook,"Sylvia Beach, Logbooks 1919–1941, Sylvia Beach...",NaN,NaN
13686,Subscription,1933-03-02,NaN,https://shakespeareandco.princeton.edu/members...,Stanislas Pascal Franchot,"Franchot, Stanislas Pascal",NaN,50.0,NaN,31.0,...,NaN,NaN,NaN,NaN,NaN,NaN,Logbook;Lending Library Card,"Sylvia Beach, Logbooks 1919–1941, Sylvia Beach...",;https://figgy.princeton.edu/concern/scanned_r...,;https://iiif.princeton.edu/loris/figgy_prod/7...


In [9]:
# omit events with unknown end date since we can't recalculate duration
# (duration in the dataset is based on the subscription duration)
subs_duration = subs_duration[subs_duration.end_date.notna()]

In [10]:
# add a new field for duration as calculated by undate
subs_duration["undate_duration"] = subs_duration.apply(lambda row: undate_duration(str(row.start_date), str(row.end_date)), axis=1)
subs_duration.head()

,event_type,start_date,end_date,member_uris,member_names,member_sort_names,subscription_price_paid,subscription_deposit,subscription_duration,subscription_duration_days,...,item_title,item_volume,item_authors,item_year,item_notes,source_type,source_citation,source_manifest,source_image,undate_duration
28,Subscription,1927,1928,https://shakespeareandco.princeton.edu/members...,Arthur Elliott Felkin,"Felkin, Arthur Elliott",NaN,NaN,1 year,365.0,...,NaN,NaN,NaN,NaN,NaN,Address Book,"Sylvia Beach, Address Book 1919–1935, box 69, ...",NaN,NaN,730
70,Subscription,1931,1932,https://shakespeareandco.princeton.edu/members...,Geraldine Deknatel;William Deknatel,"Deknatel, Geraldine;Deknatel, William",NaN,NaN,1 year,365.0,...,NaN,NaN,NaN,NaN,NaN,Address Book,"Sylvia Beach, Address Book 1919–1935, box 69, ...",NaN,NaN,730
233,Subscription,1921-07,1921-08,https://shakespeareandco.princeton.edu/members...,Mrs. G. S. Madam,"Madam, Mrs. G. S.",NaN,NaN,1 month,31.0,...,NaN,NaN,NaN,NaN,NaN,Address Book,"Sylvia Beach, Address Book 1919–1935, box 69, ...",NaN,NaN,61
234,Subscription,1921-09,1922-02,https://shakespeareandco.princeton.edu/members...,Anne Moderwell;Hiram Moderwell / H. K. Moderwell,"Moderwell, Anne;Moderwell, Hiram",NaN,NaN,5 months,153.0,...,NaN,NaN,NaN,NaN,NaN,Address Book,"Sylvia Beach, Address Book 1919–1935, box 69, ...",NaN,NaN,180
260,Subscription,1923-06,1923-10,https://shakespeareandco.princeton.edu/members...,Victor Llona,"Llona, Victor",NaN,NaN,4 months,122.0,...,NaN,NaN,NaN,NaN,NaN,Address Book,"Sylvia Beach, Address Book 1919–1935, box 69, ...",NaN,NaN,152


In [11]:
# compare undate duration with dataset duration
# limit to fields we care about
subs_duration = subs_duration[['start_date', 'end_date', 'subscription_duration', 'subscription_duration_days', 'undate_duration']]
subs_duration

,start_date,end_date,subscription_duration,subscription_duration_days,undate_duration
28,1927,1928,1 year,365.0,730
70,1931,1932,1 year,365.0,730
233,1921-07,1921-08,1 month,31.0,61
234,1921-09,1922-02,5 months,153.0,180
260,1923-06,1923-10,4 months,122.0,152
...,...,...,...,...,...
35114,1941-11-24,1941-12-24,1 month,30.0,30
35115,1941-11-24,1941-12-24,1 month,30.0,30
35116,1941-12-04,1942-01-04,1 month,31.0,31
35118,1941-12-08,1942-03-08,3 months,90.0,90


In [12]:
# what's the difference between the two?
subs_duration['duration_diff'] = subs_duration.apply(lambda row: row.undate_duration - row.subscription_duration_days, axis=1)
subs_duration

,start_date,end_date,subscription_duration,subscription_duration_days,undate_duration,duration_diff
28,1927,1928,1 year,365.0,730,365.0
70,1931,1932,1 year,365.0,730,365.0
233,1921-07,1921-08,1 month,31.0,61,30.0
234,1921-09,1922-02,5 months,153.0,180,27.0
260,1923-06,1923-10,4 months,122.0,152,30.0
...,...,...,...,...,...,...
35114,1941-11-24,1941-12-24,1 month,30.0,30,0.0
35115,1941-11-24,1941-12-24,1 month,30.0,30,0.0
35116,1941-12-04,1942-01-04,1 month,31.0,31,0.0
35118,1941-12-08,1942-03-08,3 months,90.0,90,0.0


In [13]:
subs_duration['duration_diff'].value_counts()

 0.0      9065
 30.0       30
 29.0       21
 1.0        10
-1.0         9
 28.0        4
 365.0       2
 27.0        1
 2.0         1
-3.0         1
Name: duration_diff, dtype: int64

### investigate discrepancies

In [15]:
# investigate the ones with larger differences
subset_subdurations = subs_duration[subs_duration.duration_diff != 0]
subset_subdurations

,start_date,end_date,subscription_duration,subscription_duration_days,undate_duration,duration_diff
28,1927,1928,1 year,365.0,730,365.0
70,1931,1932,1 year,365.0,730,365.0
233,1921-07,1921-08,1 month,31.0,61,30.0
234,1921-09,1922-02,5 months,153.0,180,27.0
260,1923-06,1923-10,4 months,122.0,152,30.0
...,...,...,...,...,...,...
33878,1933-12-01,1934-01-01,1 month,30.0,31,1.0
33880,1933-12-02,1934-01-02,1 month,30.0,31,1.0
33902,1934-01-02,1934-07-02,6 months,182.0,181,-1.0
33936,1934-06-02,1934-12-02,6 months,184.0,183,-1.0


In [16]:
# too many to lok at once, can we segment by subscription duration?
subset_subdurations.subscription_duration.value_counts()

1 month      38
3 months     12
2 months      7
6 months      6
4 months      5
5 months      3
1 year        2
7 months      2
8 months      2
11 months     1
10 months     1
Name: subscription_duration, dtype: int64

In [17]:
# lots of one-month subscriptions, what do the discrepancies look like?
subset_subdurations[subset_subdurations.subscription_duration == '1 month']

,start_date,end_date,subscription_duration,subscription_duration_days,undate_duration,duration_diff
233,1921-07,1921-08,1 month,31.0,61,30.0
261,1923-08,1923-09,1 month,31.0,60,29.0
271,1924-02,1924-03,1 month,29.0,59,30.0
313,1926-11,1926-12,1 month,30.0,60,30.0
354,1928-02,1928-03,1 month,29.0,59,30.0
356,1928-02,1928-03,1 month,29.0,59,30.0
393,1929-08,1929-09,1 month,31.0,60,29.0
394,1929-08,1929-09,1 month,31.0,60,29.0
430,1930-05,1930-06,1 month,31.0,60,29.0
444,1930-11,1930-12,1 month,30.0,60,30.0


The first set of these are calculated differently because they are partial dates; undate logic calculates based on earliest possible date through last possible date, but we have additional information in these cases that is project-specific and undate can't take into account, i.e. subscription duration is one month starting sometime in a known year or month.

The handful towards the end that are off by one in either direction (+/-) are a little more concerning... (potential bug in S&co code? or value calculated based on known semantic duration?)

In [18]:
# durations other than one month
subset_subdurations[subset_subdurations.subscription_duration != '1 month']

,start_date,end_date,subscription_duration,subscription_duration_days,undate_duration,duration_diff
28,1927,1928,1 year,365.0,730,365.0
70,1931,1932,1 year,365.0,730,365.0
234,1921-09,1922-02,5 months,153.0,180,27.0
260,1923-06,1923-10,4 months,122.0,152,30.0
272,1924-02,1924-04,2 months,60.0,89,29.0
293,1926-03,1926-10,7 months,214.0,244,30.0
321,1927-03,1928-02,11 months,337.0,365,28.0
331,1927-07,1927-10,3 months,92.0,122,30.0
337,1927-10,1928-06,8 months,244.0,273,29.0
349,1928-01,1928-04,3 months,91.0,120,29.0


## borrow events

compare borrow events with known duration

In [19]:
borrow_duration = events_df[events_df.borrow_duration_days.notna()]
# limit to fields we care about for this check
borrow_duration = borrow_duration[['start_date', 'end_date', 'borrow_duration_days']]
borrow_duration.head()

,start_date,end_date,borrow_duration_days
602,--01-07,--01-13,6.0
603,--01-12,--01-20,8.0
604,--01-16,--02-16,31.0
605,--01-19,--01-24,5.0
606,--01-20,--01-28,8.0


In [20]:
borrow_duration.tail()

,start_date,end_date,borrow_duration_days
29903,1961-06-30,1961-10-04,96.0
29904,1961-06-30,1961-10-04,96.0
29905,1961-06-30,1961-10-04,96.0
29907,1961-10-04,1962-03-21,168.0
29908,1961-10-04,1962-03-21,168.0


In [21]:
# add a new field for duration as calculated by undate
borrow_duration["undate_duration"] = borrow_duration.apply(lambda row: undate_duration(str(row.start_date), str(row.end_date)), axis=1)
borrow_duration.head(10)

,start_date,end_date,borrow_duration_days,undate_duration
602,--01-07,--01-13,6.0,6
603,--01-12,--01-20,8.0,8
604,--01-16,--02-16,31.0,31
605,--01-19,--01-24,5.0,5
606,--01-20,--01-28,8.0,8
607,--01-24,--03-20,55.0,55
608,--01-24,--03-20,55.0,55
609,--01-24,--03-20,55.0,55
610,--01-24,--05-30,126.0,126
611,--01-24,--05-30,126.0,126


In [22]:
# what's the difference between the two?
borrow_duration['duration_diff'] = borrow_duration.apply(lambda row: row.undate_duration - row.borrow_duration_days, axis=1)
borrow_duration

,start_date,end_date,borrow_duration_days,undate_duration,duration_diff
602,--01-07,--01-13,6.0,6,0.0
603,--01-12,--01-20,8.0,8,0.0
604,--01-16,--02-16,31.0,31,0.0
605,--01-19,--01-24,5.0,5,0.0
606,--01-20,--01-28,8.0,8,0.0
...,...,...,...,...,...
29903,1961-06-30,1961-10-04,96.0,96,0.0
29904,1961-06-30,1961-10-04,96.0,96,0.0
29905,1961-06-30,1961-10-04,96.0,96,0.0
29907,1961-10-04,1962-03-21,168.0,168,0.0


In [23]:
# what do the duration differences look like?
borrow_duration.duration_diff.value_counts()

0.0    19728
Name: duration_diff, dtype: int64

Woohoo, everything matches! 🎉

In a previous run, there were two borrow events where the calculation did not match; this was due to an error in undate duration method when the start and end dates have unknown years and dates wrap to the following year (e.g., december to january), which has now been corrected.

**Note:** One of those events has a range (--06-07/--06-06) that looks like a data error in S&co, but the data matches what is [written on the lending card](https://shakespeareandco.princeton.edu/members/davet-yvonne/cards/cf96d38f-e651-491c-a575-131ea32ce425/#).

In [24]:
borrow_duration[borrow_duration.duration_diff != 0]

,start_date,end_date,borrow_duration_days,undate_duration,duration_diff
